In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df= pd.read_csv('/content/DFORE350D1.csv', low_memory=False)

In [ ]:
df.columns

Index(['Reporting_level', 'vendor', 'as400id', 'location_id', 'Location_Name',
       'Branch_Id', 'Branch_Name', 'state', 'room_id', 'machine_type',
       'TimeSinceFirstVend', 'TimeSinceLastTxn', 'Tenure_Flag',
       'Txn_Avg_Machine_Cnt', 'Room_Open_Hours', 'Daily_Max_Room_Vend_Hours',
       'Base_Vends', 'Base_Vend_Amount', 'Top_Up_Vends', 'Top_Up_Vend_Amount',
       'Total_Vend_Hours', 'Total_Hours_Open', 'Overall_Utilization',
       'Max_Daily_Vends', 'Daily_Peak_Utilization',
       'Full_Room_Util_Gross_Hours', 'Full_Room_Util_Vend_PCT',
       'Full_Room_Util_Total_PCT', 'Avg_Daily_Vends', 'VDPM', 'Vend_Price',
       'AvgUtil_M', 'AvgUtil_T', 'AvgUtil_W', 'AvgUtil_Th', 'AvgUtil_F',
       'AvgUtil_Sa', 'AvgUtil_Su', 'nrpm', 'ContractExpirationDate',
       'Days_To_ContractExpiration'],
      dtype='object')

In [ ]:
df=df.loc[:, ['vendor', 'as400id', 'room_id', 'machine_type',
       'TimeSinceFirstVend', 'TimeSinceLastTxn', 'Tenure_Flag', 'Txn_Avg_Machine_Cnt',
       'Room_Open_Hours', 'Daily_Max_Room_Vend_Hours', 'Base_Vends',
       'Base_Vend_Amount', 'Top_Up_Vends', 'Top_Up_Vend_Amount',
       'Total_Vend_Hours', 'Total_Hours_Open', 'Overall_Utilization',
       'Max_Daily_Vends', 'Daily_Peak_Utilization',
       'Full_Room_Util_Gross_Hours', 'Full_Room_Util_Vend_PCT',
       'Full_Room_Util_Total_PCT', 'Avg_Daily_Vends', 'VDPM', 'Vend_Price',
       'AvgUtil_M', 'AvgUtil_T', 'AvgUtil_W', 'AvgUtil_Th', 'AvgUtil_F',
       'AvgUtil_Sa', 'AvgUtil_Su', 'nrpm', 'ContractExpirationDate',
       'Days_To_ContractExpiration']].dropna()

# Machines

In [ ]:
# pd.merge(pd.Series(df.room_id.unique()), df.loc[df.machine_type=='WASHER', 'room_id'], how='left', on= 'room_id')

In [ ]:
df.loc[df.machine_type=='WASHER', 'room_id'].nunique()

48947

In [ ]:
df.loc[df.machine_type=='DRYER', 'room_id'].nunique()

48650

In [ ]:
df.loc[df.machine_type=='STACKED', 'room_id'].nunique()

900

In [ ]:

df[['Overall_Utilization', 'Daily_Peak_Utilization', 'nrpm']].describe(percentiles=[.01, .99])

,Overall_Utilization,Daily_Peak_Utilization,nrpm
count,98559.000000,98559.000000,98559.000000
mean,5.091718,23.227108,38.395744
std,3.199015,10.399157,26.178304
min,0.020000,2.291663,-360.920000
1%,0.180000,7.352941,0.000000
50%,4.590000,21.111110,35.960000
99%,15.040000,57.280680,110.140000
max,47.960000,100.000000,802.510000


In [ ]:
otl= ['Overall_Utilization', 'Daily_Peak_Utilization', 'nrpm']

In [ ]:
for o in otl:
  p1=df.loc[:,o].describe(percentiles=[.01,.25,.50,.75,.99]).loc['1%']
  p99= df.loc[:,o].describe(percentiles=[.01,.25,.50,.75,.99]).loc['99%']
  df[o] = np.where(df[o] <p1, p1,df[o])
  df[o] = np.where(df[o] >p99, p99,df[o])

In [ ]:
thres= [.9, .8, .7]
threspct=['90%', '80%', '70%']

In [ ]:
cut= dict(zip(otl, list(zip(thres, threspct))))

In [ ]:
cut

{'Daily_Peak_Utilization': (0.8, '80%'),
 'Overall_Utilization': (0.9, '90%'),
 'nrpm': (0.7, '70%')}

In [ ]:
df['mc_bkt'] = np.where(df.Txn_Avg_Machine_Cnt >= 4, 1, 0)

In [ ]:
p= df.Daily_Peak_Utilization.describe(percentiles=[.8])['80%']
df['dpu_bkt']= np.where(df['Daily_Peak_Utilization'] <p, 1,0)

In [ ]:
p= df.Overall_Utilization.describe(percentiles=[.9])['90%']
df['ou_bkt']= np.where(df['Overall_Utilization'] <p, 1,0)

In [ ]:
p= df.nrpm.describe(percentiles=[.7])['70%']
df['nrpm_bkt']= np.where(df['nrpm'] <p, 1,0)

In [ ]:
df['comp_bkt']= df.mc_bkt+ df.dpu_bkt+ df.ou_bkt+ df.nrpm_bkt

In [ ]:
df['new_Overall_Utilization']= np.where(df.comp_bkt==4, 4*df.Overall_Utilization/3, df.Overall_Utilization)

In [ ]:
df['new_machine_count']= np.where(df.comp_bkt==4, 3*df.Txn_Avg_Machine_Cnt/4, df.Txn_Avg_Machine_Cnt)

In [ ]:
df.groupby(['vendor', 'machine_type']).agg({'room_id':'nunique', 'Txn_Avg_Machine_Cnt': 'sum', 'Overall_Utilization': 'mean','new_machine_count': 'sum',\
                                            'new_Overall_Utilization': 'mean'})

room_id  ...  new_Overall_Utilization
vendor machine_type           ...                         
CSCGo  DRYER           16735  ...                 4.980042
       STACKED           127  ...                 2.345564
       WASHER          16834  ...                 4.027117
DI     DRYER           31915  ...                 6.255194
       STACKED           773  ...                 6.820009
       WASHER          32113  ...                 4.782746

[6 rows x 5 columns]

# High Level Stats

In [ ]:
df.groupby(['vendor']).room_id.nunique()

vendor
CSCGo    17149
DI       32715
Name: room_id, dtype: int64

In [ ]:
pd.crosstab(df.vendor, df.machine_type, df.Txn_Avg_Machine_Cnt, aggfunc='sum')

machine_type,DRYER,STACKED,WASHER
vendor,,,
CSCGo,38661,154,39125
DI,74907,1108,76637


In [ ]:
pd.crosstab(df.vendor, df.Tenure_Flag, df.Txn_Avg_Machine_Cnt, aggfunc='sum')

Tenure_Flag,New,Tenured
vendor,,
CSCGo,6034,71906
DI,1214,151438


In [ ]:
df['UtilTimeFrame']= (df.TimeSinceFirstVend- df.TimeSinceLastTxn)/(df.TimeSinceFirstVend+1)

In [ ]:
df['UtilTimeBkt']= pd.cut(df.UtilTimeFrame, [0,.2,1])

In [ ]:
pd.qcut(df.UtilTimeFrame, 5)

0           (0.9833, 0.9834]
1           (0.9778, 0.9833]
2           (0.9833, 0.9834]
3           (0.9833, 0.9834]
4           (0.9778, 0.9833]
                 ...        
102398      (0.9778, 0.9833]
102399    (0.005425, 0.9595]
102400      (0.9833, 0.9834]
102401    (0.005425, 0.9595]
102402      (0.9595, 0.9778]
Name: UtilTimeFrame, Length: 98559, dtype: category
Categories (5, interval[float64, right]): [(0.005425, 0.9595] < (0.9595, 0.9778] < (0.9778, 0.9833] <
                                           (0.9833, 0.9834] < (0.9834, 0.989]]

In [ ]:
pd.crosstab(index= [df.vendor, df.Tenure_Flag], columns= df.UtilTimeBkt, values= df.Txn_Avg_Machine_Cnt, aggfunc='sum')

UtilTimeBkt         (0.0, 0.2]  (0.2, 1.0]
vendor Tenure_Flag                        
CSCGo  New                 456        5578
       Tenured            2457       69449
DI     New                  86        1128
       Tenured             994      150444

In [ ]:
df.head()

,vendor,as400id,room_id,machine_type,TimeSinceFirstVend,TimeSinceLastTxn,Tenure_Flag,Txn_Avg_Machine_Cnt,Room_Open_Hours,Daily_Max_Room_Vend_Hours,Base_Vends,Base_Vend_Amount,Top_Up_Vends,Top_Up_Vend_Amount,Total_Vend_Hours,Total_Hours_Open,Overall_Utilization,Max_Daily_Vends,Daily_Peak_Utilization,Full_Room_Util_Gross_Hours,Full_Room_Util_Vend_PCT,Full_Room_Util_Total_PCT,Avg_Daily_Vends,VDPM,Vend_Price,AvgUtil_M,AvgUtil_T,AvgUtil_W,AvgUtil_Th,AvgUtil_F,AvgUtil_Sa,AvgUtil_Su,nrpm,ContractExpirationDate,Days_To_ContractExpiration,mc_bkt,dpu_bkt,ou_bkt,nrpm_bkt,comp_bkt,new_Overall_Utilization,new_machine_count,UtilTimeFrame,UtilTimeBkt
0,DI,7211435,DI-5110-1,WASHER,180,2,Tenured,2,24,7.550000,527,867.50,0,0.0,325.549956,8640,3.77,11,15.729167,136.25,41.85,1.58,2.93,146.60750,1.646110,3.6126,4.5432,4.1786,4.0131,4.5846,3.5999,4.3984,43.26,2022-06-29 00:00:00.000,124.0,0,1,1,1,3,3.77,2.0,0.983425,"(0.2, 1.0]"
1,DI,18039,DI-3316-27,WASHER,180,3,Tenured,1,11,1.833333,75,166.75,0,0.0,49.683312,1980,2.51,3,16.666664,70.00,100.00,2.51,0.42,28.18075,2.223333,3.4848,3.1490,2.9567,1.3621,3.7419,4.4166,3.7833,25.77,2027-08-24 00:00:00.000,2006.0,0,1,1,1,3,2.51,1.0,0.977901,"(0.2, 1.0]"
2,DI,4424427,DI-693-76,DRYER,180,2,Tenured,1,24,5.266666,213,266.25,0,0.0,163.066662,4320,3.77,7,21.944442,215.25,100.00,3.77,1.18,44.99625,1.250000,9.2387,3.1784,9.5432,10.0480,4.9267,4.7133,10.9333,27.16,2027-01-10 00:00:00.000,1780.0,0,1,1,1,3,3.77,1.0,0.983425,"(0.2, 1.0]"
3,DI,1101216,DI-16051-11,DRYER,180,2,Tenured,1,14,7.616666,316,509.50,0,0.0,304.166636,2520,12.07,8,54.404757,363.00,100.00,12.07,1.76,86.10550,1.612341,17.6107,12.3183,13.4472,15.0183,15.2021,15.0296,17.6133,30.21,2029-07-12 00:00:00.000,2694.0,0,0,0,1,1,12.07,1.0,0.983425,"(0.2, 1.0]"
4,DI,32691,DI-5923-11,WASHER,180,3,Tenured,2,17,9.000000,699,1326.20,0,0.0,420.483286,6120,6.87,15,26.470588,173.25,41.20,2.83,3.88,224.12780,1.897281,8.0473,6.3120,7.0332,6.3634,7.8585,9.6749,10.4235,39.56,2019-07-25 00:00:00.000,-946.0,0,1,1,1,3,6.87,2.0,0.977901,"(0.2, 1.0]"
